In [1]:
from shared import get_clean_data

#DATA_FILENAME = '/home/sami/py-graph/facebook_combined.txt.gz'
#DATA_FILENAME = '/home/sami/py-graph/fb_simple.txt.gz'
DATA_FILENAME = '/home/sami/py-graph/exampledata.txt.gz'

print("Normalizing data (or loaded pre-computed)")
%time edges, num_edges, num_nodes = get_clean_data(DATA_FILENAME, shuffle=False, save_readable=True)
print('\nEDGES SHAPE: {}'.format(edges.shape))

Normalizing data (or loaded pre-computed)
Parsing from zip. Will write to file /home/sami/py-graph/exampledata.txt-cleaned.npy
ORIGINAL DIST: 1.625 MIN: 0 MAX: 8
CPU times: user 4.03 ms, sys: 0 ns, total: 4.03 ms
Wall time: 5.56 ms

EDGES SHAPE: (16, 2)


In [2]:
%load_ext Cython
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
%%cython
import numpy as np
cimport cython

cdef int UNMAPPED = -1

def linear_deterministic_greedy(int[:,::] edges,
                                int num_nodes,
                                int num_partitions,
                                int[::] partition):
    """
    This algorithm favors a cluster if it has many neighbors of a node, but
    penalizes the cluster if it is close to capacity.
    
    edges: An [:,2] array of edges.
    num_nodes: The number of nodes in the graph.
    num_partitions: How many partitions we are breaking the graph into.
    partition: The partition from a previous run. Used for restreaming.

    Returns: A new partition.
    """
    # The output partition

    if partition is None:
        partition = np.repeat(np.int32(UNMAPPED), num_nodes)
    elif len(partition) < num_nodes:
        print("THIS IS BAD AND MAY BREAK")
        print("len(partitions) = {}, num_nodes = {}".format(len(partition),num_nodes))

    cdef int[::] partition_sizes = np.zeros(num_partitions, dtype=np.int32)
         
    cdef int[::] partition_votes = np.zeros(num_partitions, dtype=np.int32)
    
    # Fine to be a little off, to stay integers
    cdef int partition_capacity = num_nodes / num_partitions
    
    cdef int last_left = edges[0,0]
    cdef int i = 0
    cdef int left = 0
    cdef int right = 0
    cdef int arg = 0
    cdef int max_arg = 0
    cdef int max_val = 0
    cdef int val = 0
    cdef int len_edges = len(edges)
    
    import time

    for i in range(len_edges):
        left = edges[i,0]
        right = edges[i,1]
    
        if last_left != left:
            # We have found a new node so assign last_left to a partition

            # Calculate available space in each partition, multiply that by partition_votes to get max_val
            #max_arg: most likely partition asssignment, ie. the partition with enough space available
            #         and the highest number of edges (relationships to other nodes)
            #max_val: current highest value of votes against remaining capacity
            #partition_votes: the number of right nodes in a partition related to current left node

            print("Assigning node {}".format(last_left))
            
            max_arg = 0
            max_val = (partition_votes[0]) * (
                       partition_capacity - partition_sizes[0])
            print("\tP{} = {} x ({} - {}) = {}".format(0,
                                                       partition_votes[0],
                                                       partition_capacity,
                                                       partition_sizes[0],
                                                       max_val))

            for arg in range(1, num_partitions):
                val = (partition_votes[arg]) * (
                       partition_capacity - partition_sizes[arg])
                
                print("\tP{} = {} x ({} - {}) = {}".format(arg,
                                                           partition_votes[arg],
                                                           partition_capacity,
                                                           partition_sizes[arg],
                                                           val))
                if val > max_val:
                    max_arg = arg
                    max_val = val

            if max_val == 0:
                max_arg = arg
                # No neighbors (or multiple maxed out) so "randomly" select
                # the smallest partition
                for arg in range(i % num_partitions, num_partitions):
                    if partition_sizes[arg] < partition_capacity:
                        max_arg = arg
                        max_val = 1
                        break
                if max_val == 0:
                    for arg in range(0, i % num_partitions):
                        if partition_sizes[arg] < partition_capacity:
                            max_arg = arg
                            break


                            
            partition_sizes[max_arg] += 1
            partition[last_left] = max_arg
            partition_votes[:] = 0
            last_left = left

            print("\tmax_arg: {}".format(max_arg))
            print("\tmax_val: {}".format(max_val))
            print(np.asarray(partition))
            
        if partition[right] != UNMAPPED:
            partition_votes[partition[right]] += 1

    # Clean up the last assignment
    max_arg = 0
    max_val = 0
    print("last one")
    for arg in range(0, num_partitions):
        if partition_sizes[arg] < partition_capacity:
            

            
            val = (partition_votes[arg]) * (
                    1 - partition_sizes[arg] / partition_capacity)
            
            print("\tP{} = {} x (1 - {} / {}) = {}".format(arg,
                                         partition_votes[arg],
                                         partition_sizes[0],
                                         partition_capacity,
                                         val))
            
            if val > max_val:
                max_arg = arg
                max_val = val


    partition[left] = max_arg

    return np.asarray(partition)

In [4]:
def draw_figures(waste_values, edge_score_values, num_partitions):
    fig, axes = plt.subplots(nrows=1, ncols=2)
    
    (waste_axes, score_axes) = axes
    waste_axes.plot(waste_values, 'r')
    waste_axes.set_xlabel('Round')
    waste_axes.set_ylabel('Number of wasted bins')
    waste_axes.set_ylim(bottom=0)
    waste_axes.set_title('Wasted Bins: {} partitions'.format(num_partitions))
    
    score_axes.plot(edge_score_values, 'r')
    score_axes.set_xlabel('Round')
    score_axes.set_ylabel('Ratio of edges crossing partitions')
    score_axes.set_ylim(bottom=0)
    score_axes.set_title('Score: {} partitions'.format(num_partitions))
    
    fig.tight_layout()

In [5]:
from shared import score


def run_restreaming_greedy(edges, num_nodes, num_partitions=10, num_iterations=10):
    print('\n{} PARTITIONS'.format(num_partitions))
    assignments = None
    print('ROUND\tWASTE\tSCORE\tCUT')
    waste_values = []
    edge_score_values = []
    flipped_edges = numpy.flipud(edges).copy()

    for i in range(num_iterations):
        assignments = linear_deterministic_greedy(edges, num_nodes, num_partitions, assignments)
        (waste, edge_score, edges_cut) = score(assignments, edges)
        waste_values.append(waste)
        edge_score_values.append(edge_score)
        print('{}\t{:0.3f}\t{:0.3f}\t{}'.format(i, waste, edge_score, edges_cut))

    draw_figures(waste_values, edge_score_values, num_partitions)
    return assignments
    
def run_restreaming_greedy_stepped(edges, node):
    # XXX: this can certainly be optimised using memoryviews
    
    # initialise left node to first item in edge list
    left = edges[0][0]
    stepped_edges = []
    count = 0
    node += 1

    for e in edges:
        if left != e[0]:
            left = e[0]

        if left > node:
            break
            
        stepped_edges.append(e)
        count += 1

    m = max([x[0] for x in stepped_edges])
    #stepped_edges = [[x[0],x[1]] for x in stepped_edges if x[1] <= m]
    if not stepped_edges:
        return (np.zeros((1, 2), dtype=np.int32), 0)
    
    #num_nodes = max([x[0] for x in stepped_edges] + [x[1] for x in stepped_edges]) + 1
    as_array = np.zeros((len(stepped_edges), 2), dtype=np.int32)
    for (i, (n_0, n_1)) in enumerate(stepped_edges):
        as_array[i,0] = n_0
        as_array[i,1] = n_1
    return (as_array, num_nodes)

In [6]:
#print(edges)
#print(num_nodes)

#assignments = run_restreaming_greedy(edges, num_nodes, num_partitions=4, num_iterations=2)

#print(assignments)

In [7]:
# Use the greedy method to break into 4 partitions
#%time run_restreaming_greedy(edges, num_nodes, num_partitions=4, num_iterations=15)

import networkx as nx
G = nx.Graph()

fn = set([x[0] for x in edges])
file_nodes = []
for f in fn:
    file_nodes.append(f)

def line_print(assignments):
    for i in range(0, len(assignments)):
        for b in range(i, len(assignments)):
            if assignments[b] != -1:
                break
        if b != len(assignments)-1:
            print("{} ".format(assignments[i]), end='')
    print()
    
assignments = np.repeat(np.int32(-1), num_nodes)
for node in file_nodes:
    assignments = np.repeat(np.int32(-1), num_nodes)
    
    e, num_nodes = run_restreaming_greedy_stepped(edges, node)
    G.add_edges_from(e)
    assignments = linear_deterministic_greedy(e, G.number_of_nodes(), 4, assignments)

    print("NODE:{}".format(node))
    #print(assignments)
    line_print(assignments)


# write to file
with open("assignments.txt", "w") as f:
    j = 0
    for a in assignments:
        f.write("{} {}\n".format(j,a))
        j += 1

Assigning node 0
	P0 = 0 x (1 - 0) = 0
	P1 = 0 x (1 - 0) = 0
	P2 = 0 x (1 - 0) = 0
	P3 = 0 x (1 - 0) = 0
	max_arg: 3
	max_val: 1
[ 3 -1 -1 -1 -1 -1 -1 -1 -1]
last one
	P0 = 0 x (1 - 0 / 1) = 0
	P1 = 0 x (1 - 0 / 1) = 0
	P2 = 0 x (1 - 0 / 1) = 0
NODE:0
3 0 
Assigning node 0
	P0 = 0 x (1 - 0) = 0
	P1 = 0 x (1 - 0) = 0
	P2 = 0 x (1 - 0) = 0
	P3 = 0 x (1 - 0) = 0
	max_arg: 3
	max_val: 1
[ 3 -1 -1 -1 -1 -1 -1 -1 -1]
Assigning node 1
	P0 = 0 x (1 - 0) = 0
	P1 = 0 x (1 - 0) = 0
	P2 = 0 x (1 - 0) = 0
	P3 = 1 x (1 - 1) = 0
	max_arg: 1
	max_val: 1
[ 3  1 -1 -1 -1 -1 -1 -1 -1]
last one
	P0 = 0 x (1 - 0 / 1) = 0
	P2 = 0 x (1 - 0 / 1) = 0
NODE:1
3 1 0 
Assigning node 0
	P0 = 0 x (1 - 0) = 0
	P1 = 0 x (1 - 0) = 0
	P2 = 0 x (1 - 0) = 0
	P3 = 0 x (1 - 0) = 0
	max_arg: 3
	max_val: 1
[ 3 -1 -1 -1 -1 -1 -1 -1 -1]
Assigning node 1
	P0 = 0 x (1 - 0) = 0
	P1 = 0 x (1 - 0) = 0
	P2 = 0 x (1 - 0) = 0
	P3 = 1 x (1 - 1) = 0
	max_arg: 1
	max_val: 1
[ 3  1 -1 -1 -1 -1 -1 -1 -1]
Assigning node 2
	P0 = 0 x (1 - 0) =

In [7]:
# Use the greedy method to break into 20 partitions
%time run_restreaming_greedy(edges, num_nodes, num_partitions=40, num_iterations=15)


40 PARTITIONS
ROUND	WASTE	SCORE	CUT


ValueError: The first argument of bincount must be non-negative